In [7]:
import methods

In [1]:
from ctypes import *
import os
import numpy as np


class bench_result(Structure):
    _fields_ = [
        ("original_size", c_size_t),
        ("compressed_size", c_size_t),
        ("compression_time", c_double),
        ("decompression_time", c_double),
        ("max_error", c_double),
        ("mean_absolute_error", c_double),
    ]

    # https://stackoverflow.com/a/62011887/3901677
    def __repr__(self) -> str:
        values = ", ".join(f"{name}={value}" for name, value in self._asdict().items())
        return f"<{self.__class__.__name__}: {values}>"

    def _asdict(self) -> dict:
        return {field[0]: getattr(self, field[0]) for field in self._fields_}


lib = CDLL(os.path.abspath("../build/libcompression-benchmark.so"))
lib.recompress.argtypes = [POINTER(bench_result), c_char, c_void_p, c_int]
lib.recompress.restype = c_int


arr = np.array([1, 2, 3, 1, 5], dtype="double")
buf = arr.tobytes()
results = bench_result()
ret = lib.recompress(byref(results), ord("d"), buf, arr.size)
arr2 = np.frombuffer(buf, dtype="double")
print(arr2)
print(results)


Using method Sz3
Compressing... done
Decompressing... done
Comparing... done
Original size:        40
Compressed size:      153
Ratio:                382.5%
Compression time:     0.000493369s
Decompression time:   0.000250983s
Mean Absolute Error:  0.6
Max Error:            1
[1. 2. 3. 1. 5.]
<bench_result: original_size=40, compressed_size=153, compression_time=0.000493369, decompression_time=0.000250983, max_error=1.0, mean_absolute_error=0.6>
